# Statistical analysis in Python using Pingouin

In [ ]:
import numpy as np
import pandas as pd
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation

import biopsykit as bp

from IPython.display import Markdown

%load_ext autoreload
%autoreload 2
%matplotlib widget

**Pingouin**: Open source library for statistical analyses in Python, based on *pandas* and *numpy*.

Installation: `pip install pingouin`  
Link: https://pingouin-stats.org/  
Tutorial: https://pingouin-stats.org/guidelines.html

Import: `import pingouin as pg`

In [ ]:
sns.set_theme(style="ticks")
sns.set_palette(sns.color_palette("light:#003865", n_colors=4)[::-1])
plt.close('all')

# parameter for add_stat_annotation
pvalue_thres = [[1e-3, "***"], [1e-2, "**"], [0.05, "*"]]

<div class="alert alert-block alert-info">
    <b>Note:<b> Just like Seaborn, it is also advantageous for Pingouin if the data is in <i>long format<i>!
</div>

## Daten importieren

Datensatz aus Richer et al. "Tackling Acute Stress: Evaluating the Cold Face Test as a Potential Mechanism for Stress-reducing Wearables"
HR(V), Cortisol und Fragebögen während der Durchführung des MIST mit zwei Gruppen: Kontrollgruppe und Interventionsgruppe
* HR(V): Variablen Herzrate (HR), RMSSD, pNN50, SD1/SD2  pro MIST Phase (1-3) und pro MIST Subphase (Baseline (BL), Cold Face Intervention (CFI), Arithmetic Tasks (AT), Feedback (FB)), jeweils absolut und relativ (z.B. $\text{HR}_{rel}$, normalisiert pro Subject vor Beginn des Stressors)
* Fragebogen: MDBF (Multidimensionaler Bedindlichkeitsfragebogen, jeweils *pre* und *post* Stress)
* Cortisol: 7 Speichelproben an den Zeitpunkten -27 (S0, zur Kontrolle für hohe Baseline), 0, 30, 40, 50, 60, 70 (min relativ zu MIST-Start)

### HR(V)

In [ ]:
df_hr = pd.read_csv("data/hr_sample.csv", index_col=["condition", "subject", "phase", "subphase"])
df_hr.head()

### Fragebögen

In [ ]:
df_mood = pd.read_csv("data/mood_sample.csv", index_col=["condition", "subject", "time", "type"])
df_mood.head()

### Cortisol

In [ ]:
df_cort_raw = pd.read_csv("data/cortisol_sample.csv", index_col=["condition", "subject"])
saliva_times = [-27, 0, 30, 40, 50, 60, 70]

# Umwandeln ins Long-Format
df_cort_raw = pd.DataFrame(df_cort_raw.stack())
df_cort_raw.columns = ["cortisol"]
df_cort_raw.index = df_cort_raw.index.rename('sample', level=2)

# Aufteilen der Cortisol-Werte in Wert davor (Sample 1) und maximaler Wert pro Subject danach (Samples 2 bis 6)
cort_pre = df_cort_raw.unstack(level='sample').iloc[:, 1]
cort_post = df_cort_raw.unstack(level='sample').iloc[:, 2:].max(axis=1)
df_cort_inc = pd.concat([cort_pre, cort_post], axis=1, keys=['pre', 'post'], names=['time'])
df_cort_inc = pd.DataFrame(df_cort_inc.stack(), columns=["cortisol"])

# Cortisol Features berechnen
df_cort = bp.saliva.auc(df_cort_raw, saliva_times=saliva_times)
df_cort = df_cort.join(bp.saliva.max_increase(df_cort_raw, percent=True))
df_cort = df_cort.join(bp.saliva.slope(df_cort_raw, sample_idx=[1, 4], saliva_times=saliva_times))

# Feature-Dataframe ins Long-Format umwandeln
df_cort = bp.saliva.wide_to_long(df_cort)
df_cort.head()

## Reaktion auf den MIST

### Fragestellung I: Erzeugt jede Phase des MIST eine signifikante kardiovaskuläre Stressreaktion?

#### Untersuchung

Bei der *Kontrollgruppe* wird für jede MIST-Phase gesondert untersucht, ob die Herzrate zwischen den Subphasen *BL* und *AT* signifikant ansteigt.

#### Statistik

##### Analyse
* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
* *Test*: t-tests für abhängige Stichproben
* *AV*: HR
* *Innersubjektfaktor*: Subphase
* *Maß für Effektstärke*: Hedge's g

=> Analoge Analyse für die weiteren HRV-Parameter (signifikante Verringerung der HRV während Stress?)

In [ ]:
df_hr.head()

In [ ]:
dv = "HR"
subphases = ["BL", "AT"]
# Selektieren der zu untersuchenden Daten: Kontrollgruppe, alle Subjects, alle MIST Phasen, Subphasen BL und AT
df_analysis = df_hr[[dv]].loc[pd.IndexSlice["Control", :, :, subphases]]

df_analysis.reset_index()

In [ ]:
# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.normality(data=df, dv=dv, group="subphase"))

# gepaarte t-tests pro MIST-Phase
tt_phase = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.pairwise_ttests(data=df, dv=dv, within="subphase", subject="subject", effsize="hedges"))

In [ ]:
# Ausgabe
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))

display(Markdown("**Gepaarte t-tests**"))
display(tt_phase.round(3))

##### Interpretation

"Pairwise t-tests for each MIST phase individually revealed significant HR increases between *BL* and *AT* subphases indicating an effective sympathetic activation due to the stressor. Additionally, significant decreases of HRV measures occured in some, but not all, *MIST phases* (Table 2)."

#### Boxplot

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_analysis.reset_index(), x="phase", y=dv, hue="subphase")
ax.set_ylabel("HR [bpm]")

fig.tight_layout()

**Hinzufügen von Signifikanzklammern**

Mittels der Funktion `add_stat_annotation()` der Bibliothek `statannot` können den Boxplots "Signifikanzklammern" hinzugefügt werden.

Installation: `pip install statannot`  
Dokumentation: https://github.com/webermarcolivier/statannot  
Beispiele: https://github.com/webermarcolivier/statannot/blob/master/example/example.ipynb

`add_stat_annotation()` benötigt eine Liste mit Namen der Boxplot-Paare, über welche Klammern gesetzt werden sollen.  

Listenformat => Aus Dokumentation der Funktion:
* For non-grouped boxplot: `[(cat1, cat2), (cat3, cat4)]`
* For boxplot grouped by hue: `[((cat1, hue1), (cat2, hue2)), ((cat3, hue3), (cat4, hue4))]`

In [ ]:
(("MIST1", "BL"), ("MIST1", "AT"))

In [ ]:
# Signifikante MIST-Phasen
sig_list = ["MIST1", "MIST2", "MIST3"]
# Konstruieren der Boxplot-Paare-Liste
sig_pairs = [tuple([(sig, sub) for sub in subphases]) for sig in sig_list]
sig_pairs

#### Boxplot mit Signifikanzklammern

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_analysis.reset_index(), x="phase", y=dv, hue="subphase")
ax.set_ylabel("HR [bpm]")

add_stat_annotation(
    ax=ax, data=df_analysis.reset_index(), 
    x="phase", y=dv, hue="subphase", 
    box_pairs=sig_pairs,
    pvalue_thresholds=pvalue_thres,
    test="t-test_paired",
    comparisons_correction=None
)
fig.tight_layout()

#### Violinplot

In [ ]:
fig, ax = plt.subplots()
sns.violinplot(data=df_analysis.reset_index(), x="phase", y=dv, hue="subphase", split=True)
ax.set_ylabel("HR [bpm]")

fig.tight_layout()

### Fragestellung II: Erhöht sich das Stresslevel im Laufe der drei MIST Phasen?

#### Untersuchung

Bei der *Kontrollgruppe* wird untersucht, ob die Herzrate jeweils in den *BL*, *AT* bzw. *FB*-Phasen im Laufe der Zeit (über die drei MIST-Phasen) signifikant ansteigt.

#### Statistik

##### Analyse

* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
    * Test auf Varianzgleichheit (Levene-Test)
* *Test*: ANOVA mit Messwiederholung (rm-ANOVA)
* *AV*: HR, jeweils für die Subphasen *BL*, *AT* und *FB* getrennt
* *Innersubjektfaktor*: Phase
* *Maß für Effektstärke*: $\eta_p^{2}$ (partielles eta-quadrat)
* Post-Hoc-Test: t-tests für abhängige Stichproben (einseitig) mit Bonferroni-Korrektur zur Neutralisierung der Alphafehler-Kumulierung bei multiplen Vergleichen

=> Analoge Analyse für die weiteren HRV-Parameter

In [ ]:
dv = "HR"
subphases = ["BL", "AT", "FB"]
# Selektieren der zu untersuchenden Daten: Kontrollgruppe, alle Subjects, alle MIST Phasen, Subphasen BL, AT und FB
df_analysis = df_hr[[dv]].loc[pd.IndexSlice["Control", :, :, subphases]]

# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby(by=["subphase"]).apply(lambda df: pg.normality(data=df, dv=dv, group="phase"))
# Test auf Homoskedastizität
df_hom = df_analysis.reset_index().groupby(by=["subphase"]).apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="phase"))

# rm-ANOVA pro Subphase
anova_subph = df_analysis.reset_index().groupby(by=["subphase"]).apply(lambda df: pg.rm_anova(data=df, dv=dv, within="phase", subject="subject"))
# post-hoc
posthoc_subph = df_analysis.reset_index().groupby(by=["subphase"]).apply(lambda df: pg.pairwise_ttests(data=df, dv=dv, within="phase", subject="subject", effsize="hedges", tail='one-sided', padjust='bonf'))

# Ausgabe
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(Markdown("Alle Varianzen (in etwa) gleich: **{}**".format(df_hom['equal_var'].all())))

display(Markdown("**rm-ANOVA**"))
display(anova_subph.round(3))

display(Markdown("**Post-hoc Tests**"))
display(posthoc_subph.round(3))

# Optional: Filtern der signifikanten Post-Hoc-Tests
#display(posthoc_subph[posthoc_subph['p-corr'] < 0.05].round(3))

##### Interpretation

Repeated-measures *ANOVA* of HR(V) measures during *MIST subphases* indicate increasing stress levels over time, shown by a significant main effect *MIST Phase* during *BL*, as well as in *FB* and *AT* (see Table 3). Post-hoc testing of HR changes showed significant differences during *BL* between *MIST1* and *MIST3* ($t(12) = −4.285, p = 0.002, g = −0.438$) and between *MIST1* and *MIST2* ($t(12) = −2.634, p = 0.030, g = −0.384$), respectively. […]

#### Plots

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_analysis.reset_index(), x="subphase", y=dv, hue="phase")

sig_pairs = [
    (("AT", "MIST1"), ("AT", "MIST2")),
    (("AT", "MIST1"), ("AT", "MIST3")),
    (("BL", "MIST1"), ("BL", "MIST3")),
    (("BL", "MIST2"), ("BL", "MIST3")),
    (("FB", "MIST1"), ("FB", "MIST3")),
    (("FB", "MIST2"), ("FB", "MIST3")),
]

add_stat_annotation(
    ax=ax, data=df_analysis.reset_index(), 
    x="subphase", y=dv, hue="phase", 
    box_pairs=sig_pairs,
    pvalue_thresholds=pvalue_thres,
    test="t-test_paired",
    comparisons_correction=None
)

ax.set_ylabel("HR [bpm]")

fig.tight_layout()

### Fragestellung III: Verschlechtert der MIST die Stimmung?

#### Untersuchung

Bei der *Kontrollgruppe* wird untersucht, ob sich die Stimmung (gemessen mittels MDBF) als Reaktion auf den MIST signifikant verschlechtert hat.

#### Statistik

##### Analyse
* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
* *Test*: Gepaarte t-tests
* *AV*: MDBF-Skalen
* *Innersubjektfaktor*: Zeit (pre/post)
* *Maß für Effektstärke*: Hedge's g

In [ ]:
df_mood.head()

In [ ]:
#df_analysis.unstack(level='time')

In [ ]:
dv = "MDBF"
# Selektieren der zu untersuchenden Daten: Kontrollgruppe
df_analysis = df_mood.xs("Control")

# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby(by=["type"]).apply(lambda df: pg.normality(data=df, dv=dv, group="time"))

# gepaarte t-tests pro MDBF Subskala
ttests = df_analysis.reset_index().groupby(["type"]).apply(lambda df: pg.pairwise_ttests(data=df.reset_index(), within="time", dv=dv, subject="subject", effsize="hedges"))

# Berechnen der prozentualen Verschlechterung der Stimmung
df_decrease = pd.DataFrame(df_analysis.unstack(level='time').groupby("type").apply(lambda x: (x[("MDBF", 'post')] / x[("MDBF", 'pre')] - 1.0) * 100.0), columns=["decrease_percent"])
df_decrease = df_decrease.groupby("type").agg(['mean', 'std'])

# Ausgabe
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(df_norm)

display(Markdown("**Gepaarte t-tests**"))
display(ttests.round(3))

display(Markdown("**Verschlechterung der Stimmung (Mean ± SD)**"))
display(df_decrease.T.round(2))

##### Interpretation

"The stress-inducing effects of the *MIST* in the *Control* group were further confirmed by the MDBF questionnaire. Mood worsened significantly in all MDBF dimensions as indicated by pairwise t-tests (*Good-Bad*: $t(12) = 5.761, p < 0.001, g = 2.105$, decrease: $36.93\%±22.26\%~(\text{M}±\text{SD})$; *Awake-Tired*: $t(12) = 3.169, p = 0.008, g = 0.615$, decrease: $12.45\% ± 13.28\%$; *Calm-Nervous*: $t(12) = 5.094, p < 0.001, g = 1.855$, decrease: $35.44\% ± 23.92\%$)."

#### Plots

In [ ]:
fig, ax = plt.subplots()
x_order = ["good_bad", "awake_tired", "calm_nervous"]
sns.boxplot(data=df_analysis.reset_index(), x="type", y=dv, hue="time", order=x_order)

sig_pairs = [
    (("good_bad", "pre"), ("good_bad", "post")),
    (("awake_tired", "pre"), ("awake_tired", "post")),
    (("calm_nervous", "pre"), ("calm_nervous", "post")),
]

add_stat_annotation(
    ax=ax, data=df_analysis.reset_index(), 
    x="type", y=dv, hue="time", 
    order=x_order,
    box_pairs=sig_pairs,
    pvalue_thresholds=pvalue_thres,
    test="t-test_paired",
    comparisons_correction=None
)

fig.tight_layout()

### Fragestellung IV: Erzeugt der MIST eine signifikante endokrine Stressreaktion?

#### Untersuchung

Bei der *Kontrollgruppe* wird untersucht, ob sich der Cortisolspiegel vor (S1) und nach dem Stressor (S2-S6) signifikant erhöht hat.

#### Statistik

##### Analyse
* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
* *Test*: Gepaarte t-tests
* *AV*: Cortisol
* *Innersubjektfaktor*: Zeit (pre/post)
* *Maß für Effektstärke*: Hedge's g

In [ ]:
df_cort_inc.head()

In [ ]:
dv = "cortisol"
# Selektieren der zu untersuchenden Daten: Kontrollgruppe
df_analysis = df_cort_inc.xs("Control")

# Test auf Normalverteilung
df_norm = pg.normality(data=df_analysis.reset_index(), dv=dv, group="time")

# gepaarte t-tests
ttests = pg.pairwise_ttests(data=df_analysis.reset_index(), dv=dv, within="time", subject="subject", effsize="hedges")

# Berechnen des prozentualen Cortisolanstiegs
df_increase = df_analysis.unstack(level='time')
df_increase = pd.DataFrame((df_increase[("cortisol", "post")] / df_increase[("cortisol", "pre")] - 1.0) * 100, columns=["increase_percent"])

# Ausgabe
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))

display(Markdown("**Gepaarte t-tests**"))
display(ttests.round(3))

display(Markdown("**Cortisolanstieg**"))
display(df_increase.mean().round(2))

##### Interpretation

"Analysis of saliva samples showed that the *Control* group overall responded to the stressor. On average, cortisol levels increased significantly after the *MIST* ($t(12) = -2.284, p = 0.041, g = -0.623$) with an maximum increase of $75.4\%$ compared to S1."

## Effekt der Intervention auf akute Stressreaktionen

### Fragestellung I: Induziert der MIST auch in der Interventionsgruppe eine signifikante kardiovaskuläre Stressreaktion?

#### Untersuchung

Bei der *Interventionsgruppe* wird für jede MIST-Phase gesondert untersucht, ob die Herzrate zwischen den Subphasen *BL* und *AT* signifikant ansteigt.

#### Statistik

##### Analyse

* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
* *Test*: Gepaarte t-tests
* *AV*: HR
* *Innersubjektfaktor*: Subphase
* *Maß für Effektstärke*: Hedge's g (bzw. Common Language Effect Size (CLES) bei Wilcoxon-Test)

=> Analoge Analyse für die weiteren HRV-Parameter (signifikante Verringerung der HRV während Stress?)

In [ ]:
dv = "HR"
subphases = ["BL", "AT"]
# Selektieren der zu untersuchenden Daten: Kontrollgruppe, alle Subjects, alle MIST Phasen, Subphasen BL und AT
df_analysis = df_hr[[dv]].loc[pd.IndexSlice["Intervention", :, :, subphases]]

# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby("phase").apply(
    lambda df: pg.normality(data=df, dv=dv, group="subphase")
)

# Wilcoxon-Tests pro MIST-Phase
wilc_phase = df_analysis[dv].groupby("phase").apply(
    lambda df: pg.wilcoxon(
        x=df.xs("BL", level="subphase"), 
        y=df.xs("AT", level="subphase"), tail='one-sided'
    )
)

# Zum Vergleich: gepaarte t-tests pro MIST-Phase
tt_phase = df_analysis.reset_index().groupby("phase").apply(
    lambda df: pg.pairwise_ttests(
        data=df, dv=dv, within="subphase", 
        subject="subject", effsize="hedges", 
        tail='one-sided'
    )
)

# Ausgabe
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(df_norm)
display(Markdown("Normalverteilung **nicht** für **alle** Verteilungen gegeben => *Wilcoxon-Test* statt gepaartem t-test"))

display(Markdown("**Wilcoxon-Tests**"))
display(wilc_phase.round(3))
display(Markdown("**Zum Vergleich: gepaarte t-tests** (liefert vergleichbare Ergebnisse)"))
display(tt_phase.round(3))

##### Interpretation

"Even though the HR of the Intervention group was slightly decreased compared to the *Control* group during the *AT* and *FB* sub-phases both groups reached comparable stress levels especially during *MIST2* and *MIST3*. This finding is supported by significant HR increases during *AT* in each *MIST* phase for both groups (*Control*: Table 2, *Intervention*: Table 5). […] Thus, the MIST successfully caused sympathetic activation during *AT* and *FB* regardless of the condition."

### Fragestellung II: Zeigt die Intervention einen Effekt auf die kardiovaskuläre Stressreaktion?

#### Untersuchung – I

Es wird untersucht, ob die Intervention signifikante Unterschiede zwischen den beiden Gruppen während des MIST während der *BL*-Subphase verursacht.

#### Statistik

##### Analyse

* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
    * Test auf Varianzgleichheit (Levene-Test)
* *Test*: Mixed ANOVA
* *AV*: $\text{HR}_{rel}$ (normalisierte Herzrate) während *BL* => da in dieser Analyse nicht mehr ausschließlich Innersubjektfaktoren (Messwiederholung) betrachtet werden, sondern Gruppenvergleiche
* *Innersubjektfaktor*: Phase
* *Zwischensubjektfaktor*: Gruppe
* *Maß für Effektstärke*: $\eta_p^2$

=> Analoge Analyse für die weiteren HRV-Parameter

**Baseline**

In [ ]:
dv = "HR_rel"

df_analysis = df_hr.xs("BL", level='subphase')

# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
# Test auf Homoskedastizität
df_hom = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="condition"))

# Mixed ANOVA
anova_mixed = pg.mixed_anova(data=df_analysis.reset_index(), dv=dv, within="phase", between="condition", subject="subject")
# Post-hoc Tests
posthoc_mixed = pg.pairwise_ttests(data=df_analysis.reset_index(), dv=dv, within="phase", between="condition", subject="subject", effsize="hedges", tail='one-sided', padjust='bonf')

# Ausgaben
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(df_norm.round(3))
display(Markdown("Alle Varianzen (in etwa) gleich: **{}**".format(df_hom['equal_var'].all())))

display(Markdown("**Mixed ANOVA**"))
display(anova_mixed.round(3))

display(Markdown("**Post-hoc Tests**"))
display(Markdown("Signifikanter Interaktionseffekt => Betrachten der Gruppenunterschiede zu jedem Zeitpunkt (phase * condition))"))
display(posthoc_mixed.round(3))

##### Interpretation

"Over the three *MIST* phases the *HR* during *BL* of the *Intervention* group remained roughly constant, whereas it showed strong increases over time in the *Control* group (Figure 5). This drift of baseline levels in the *Control* group is reflected by a significant interaction of *condition* and *MIST Phase* in HR and (Table 6). Post-hoc tests in Table 7 revealed that baselines started to significantly differ in *MIST2* and continued for *MIST3*."

#### Plots

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_analysis.reset_index(), x="phase", y=dv, hue="condition", hue_order=["Control", "Intervention"], ax=ax)

sig_pairs = [
    (("MIST2", "Control"), ("MIST2", "Intervention")),
    (("MIST3", "Control"), ("MIST3", "Intervention")),
]

add_stat_annotation(
    ax=ax, data=df_analysis.reset_index(), 
    x="phase", y=dv, hue="condition", 
    box_pairs=sig_pairs,
    pvalue_thresholds=pvalue_thres,
    test="t-test_ind",
    hue_order=["Control", "Intervention"],
    comparisons_correction='bonferroni'
)

ax.set_title("Relative Heart Rate during BL")
ax.set_ylabel("$\Delta HR~[\%]$")
fig.tight_layout()

#### Untersuchung – II

Es wird untersucht, ob die Intervention signifikante Unterschiede zwischen den beiden Gruppen während des MIST während der *CFI*-Subphase verursacht.

#### Statistik

##### Analyse

* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
    * Test auf Varianzgleichheit (Levene-Test)
* *Test*: Mixed ANOVA
* *AV*: $\text{HR}_{rel}$ (normalisierte Herzrate) während *CFI* => da in dieser Analyse nicht mehr ausschließlich Innersubjektfaktoren (Messwiederholung) betrachtet werden, sondern Gruppenvergleiche
* *Innersubjektfaktor*: Phase
* *Zwischensubjektfaktor*: Gruppe
* *Maß für Effektstärke*: $\eta_p^2$

=> Analoge Analyse für die weiteren HRV-Parameter

In [ ]:
dv = "HR_rel"

df_analysis = df_hr.xs("CFI", level='subphase')

# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
# Test auf Homoskedastizität
df_hom = df_analysis.reset_index().groupby(by=["phase"]).apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="condition"))

# Mixed ANOVA
anova_mixed = pg.mixed_anova(data=df_analysis.reset_index(), dv=dv, within="phase", between="condition", subject="subject")

# Ausgaben
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(df_norm.round(3))
display(Markdown("Alle Varianzen (in etwa) gleich: **{}**".format(df_hom['equal_var'].all())))

display(Markdown("**Mixed ANOVA**"))
display(anova_mixed.round(3))

##### Interpretation

"For the *CFI/RP* sub-phase ANOVA revealed no significant interaction effects, but significant main effects for *condition* (Table 6). These findings indicate that the Intervention group recovered better from acute stress than the Control group."

#### Plots

In [ ]:
dv = "HR_rel"
df_plot = df_hr.loc[pd.IndexSlice[:, :, :, ["BL", "CFI"]]]
fig, axs = plt.subplots(figsize=(10, 5), ncols=2)

for (key, df), ax in zip(df_plot.reset_index().groupby("subphase"), axs):
    sns.violinplot(data=df, x="phase", y=dv, hue="condition", hue_order=["Control", "Intervention"], split=True, ax=ax)
    ax.set_title("Relative Heart Rate during {}".format(key))
    ax.set_ylabel("$\Delta HR~[\%]$")

fig.tight_layout()

### Fragestellung III: Zeigt die Intervention einen Effekt auf die Stimmungsverschlechterung?

#### Untersuchung

Es wird untersucht, ob die Intervention signifikante Unterschiede zwischen den beiden Gruppen hinsichtlich der stressinduzierten Verschlechterung der Stimmung erzeugt.

#### Statistik

##### Analyse
* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
    * Test auf Varianzgleichheit (Levene-Test)
* *Test*: Mixed ANOVA
* *AV*: MDBF-Subskalen
* *Innersubjektfaktor*: Zeit
* *Zwischensubjektfaktor*: Gruppe
* *Maß für Effektstärke*: $\eta_p^2$

In [ ]:
df_mood.head()

In [ ]:
dv = "MDBF"
# Selektieren der zu untersuchenden Daten
df_analysis = df_mood


# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby(by=["type", "time"]).apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
# Test auf Homoskedastizität
df_homo = df_analysis.reset_index().groupby(by=["type", "time"]).apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="condition"))

# Mixed ANOVA pro MDBF-Subskala
anova_mixed = df_analysis.reset_index().groupby(by=["type"]).apply(lambda df: pg.mixed_anova(data=df, dv=dv, within="time", between="condition", subject="subject"))

# Ausgabe
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(Markdown("Alle Varianzen (in etwa) gleich: **{}**".format(df_hom['equal_var'].all())))

display(Markdown("**Mixed ANOVA**"))
display(anova_mixed.round(3))

##### Interpretation

"Subjects in the Intervention group experienced slightly decreased mood worsening compared to the Control group, especially in the *Awake-Tired* dimension (Control: $12.45\% ± 13.28\%$; Intervention: $7.84\% ± 20.38\%$). However, ANOVA revealed no significant interaction effects (*condition* by *time*) and no significant main effects for *conditon*."

### Fragestellung IV: Zeigt die Intervention einen Effekt auf die endokrine Stressreaktion?

#### Untersuchung – I

Es wird untersucht, ob die Intervention signifikante Unterschiede zwischen den beiden Gruppen hinsichtlich der endokrinen Stressreaktion erzeugt.

#### Statistik

##### Analyse
* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
* *Test*: Unabhängiger t-test (beidseitig)
* *AVs*: 
    * $\text{max_inc}$ (maximaler Cortisolanstieg in Prozent)
    * $AUC_g$
    * $AUC_i$
    * $a_{S1S4}$
* *Maß für Effektstärke*: Hedge's g

=> Analog für andere Cortisol-Parameter ($\text{AUC}_g$, $\text{AUC}_i$, slope $a_{s1s4}$, ...)

In [ ]:
df_cort

In [ ]:
dv = "cortisol"
biomarker = ["max_inc_percent", "auc_g", "auc_i", "slope14"]
df_analysis = df_cort.loc[pd.IndexSlice[:, :, biomarker]]

# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby("biomarker").apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
df_norm_grp = pd.DataFrame(df_norm.reset_index().groupby("biomarker").apply(lambda df: df['normal'].all())).T

display(df_norm)
display(df_norm_grp)

# t-tests pro Biomarker
ttest_group = df_analysis.reset_index().groupby("biomarker").apply(lambda df: pg.pairwise_ttests(data=df, dv=dv, between="condition", subject="subject", effsize="hedges"))
# Mann-Whitney-U-Tests pro Biomarker
mwu_group = df_analysis[dv].groupby(by="biomarker").apply(lambda df: pg.mwu(x=df.xs("Control", level="condition"), y=df.xs("Intervention", level="condition")))

# Ausgaben
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(Markdown("Normalverteilung **nicht** für **alle** Verteilungen gegeben => *Mann-Whitney-U-Test* statt t-test bei den nicht-normalverteilten Variablen"))

for bm in biomarker:
    is_normal = df_norm_grp[bm].all()
    display(Markdown("* *{}* => Normal: **{}**".format(bm, is_normal)))
    if is_normal:
        display(ttest_group.loc[bm])
    else:
        display(mwu_group.loc[bm])

#display(Markdown("**Mann-Whitney-U-Tests**"))
#display(mwu_group.round(3))
#display(Markdown("**t-tests**"))
#display(ttest_group.round(3))

##### Interpretation

"Subjects in the Intervention group had significantly lower maximum cortisol responses compared to the Control group ($t(24) = 2.505, p = 0.019, d = 0.952$)."

#### Plots

In [ ]:
fig, axs = plt.subplots(ncols=len(biomarker), figsize=(12,5))
xlabel_map = {
    'auc_g': r"$AUC_G$",
    'auc_i': r"$AUC_I$",
    'max_inc_percent': r"$Inc_{max}$",
    'slope14': r"$a_{S1S4}$",
}
ylabel_map = {
    'auc_g': r"Cortisol AUC $\left[\frac{nmol \cdot min}{l}\right]$",
    'auc_i': r"Cortisol AUC $\left[\frac{nmol \cdot min}{l}\right]$",
    'max_inc_percent': r"Cortisol Change $[\%]$",
    'slope14': r"Cortisol Change $\left[\frac{nmol}{l \cdot min}\right]$",
}

for (key, df), ax in zip(df_analysis.reset_index().groupby("biomarker"), axs):
    sns.boxplot(data=df, x="condition", y="cortisol", ax=ax)
    ax.set_xlabel(xlabel_map[key])
    ax.set_ylabel(ylabel_map[key])
    

# stat annotation for significant features
for bm, i, test in zip(["max_inc_percent", "slope14"], [2, 3], ["t-test_ind", "Mann-Whitney"]):   
    add_stat_annotation(
        ax=axs[i], data=df_analysis.xs(bm, level="biomarker").reset_index(),
        x="condition", y=dv, 
        box_pairs=[("Control", "Intervention")],
        pvalue_thresholds=pvalue_thres,
        test=test,
        order=["Control", "Intervention"]
    )

fig.tight_layout()

#### Untersuchung – II
Es wird untersucht, ob die Intervention den Cortisolverlauf beeinflusst.

#### Statistik

##### Analyse
* Vorbereitende Analyse:
    * Test auf Normalverteilung (Shapiro-Wilk-Test)
    * Test auf Varianzgleichheit (Levene-Test)
* *Test*: Mixed ANOVA
* *AV*: Cortisol-Rohwerte
* *Maß für Effektstärke*: $\eta_p^2$

In [ ]:
dv = "cortisol"
# Entfernen von S0
df_analysis = df_cort_raw.drop('0', level='sample')

# Test auf Normalverteilung
df_norm = df_analysis.reset_index().groupby("sample").apply(lambda df: pg.normality(data=df, dv=dv, group="condition"))
# Test auf Homoskedastizität
df_hom = df_analysis.reset_index().groupby("sample").apply(lambda df: pg.homoscedasticity(data=df, dv=dv, group="condition"))

anova_mixed = pg.mixed_anova(data=df_analysis.reset_index(), dv=dv, between="condition", within="sample", subject="subject")

# Ausgaben
display(Markdown("Alle normalverteilt: **{}**".format(df_norm['normal'].all())))
display(df_norm.round(3))
display(Markdown("Alle Varianzen (in etwa) gleich: **{}**".format(df_hom['equal_var'].all())))

display(anova_mixed.round(3))

##### Interpretation

ANOVA further showed a significant interaction of *condition* by *time* for raw cortisol samples ($F(5,120) = 3.839, p = 0.003, \eta_p^2 = 0.138$).